In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
print(check_output(["ls", "../input/video_level"]).decode("utf8"))

In [ ]:
print(check_output(["ls", "../input/frame_level"]).decode("utf8"))

In [ ]:
labels_df = pd.read_csv('../input/label_names.csv')
labels_df

In [ ]:
labels_df.shape

In [ ]:
labels_df.iloc[45]

In [ ]:
submit_df = pd.read_csv('../input/sample_submission.csv')
submit_df

In [ ]:
import tensorflow as tf
labels = []
textual_labels = []
textual_labels_nested = []
filenames = ["../input/video_level/train-{}.tfrecord".format(i) for i in range(10)]
total_sample_counter = 0

label_counts = []

for filename in filenames:
    for example in tf.python_io.tf_record_iterator(filename):
        total_sample_counter += 1
        tf_example = tf.train.Example.FromString(example)

        label_example = list(tf_example.features.feature['labels'].int64_list.value)
        label_counts.append(len(label_example))
        labels = labels + label_example
        label_example_textual = list(labels_df[labels_df['label_id'].isin(label_example)]['label_name'])
        textual_labels_nested.append(set(label_example_textual))
        textual_labels = textual_labels + label_example_textual
        if len(label_example_textual) != len(label_example):
            print('label names lookup failed: {} vs {}'.format(label_example, label_example_textual))

print('label ids missing from label_names.csv: {}'.format(sorted(set(labels) - set(labels_df['label_id']))))
print('Found {} samples in all of the 10 available tfrecords'.format(total_sample_counter))

In [ ]:
tf.python_io.tf_record_iterator("../input/video_level/train-0.tfrecord")

In [ ]:
import tensorflow as tf
for example in tf.python_io.tf_record_iterator("../input/video_level/train-0.tfrecord"):
#    total_sample_counter += 1
    tf_example1 = tf.train.Example.FromString(example)
tf_example1
#print (example)

In [ ]:
tf_example1 = tf.train.Example.FromString('../input/video_level/train-0.tfrecord[0]')
tf_example1

In [ ]:
shape = tf.unstack(tf.shape(example))
shape

In [ ]:
def prepare_serialized_examples(self, serialized_example,
      max_quantized_value=2, min_quantized_value=-2):

    contexts, features = tf.parse_single_sequence_example(
        serialized_example,
        context_features={"video_id": tf.FixedLenFeature(
            [], tf.string),
                          "labels": tf.VarLenFeature(tf.int64)},
        sequence_features={
            feature_name : tf.FixedLenSequenceFeature([], dtype=tf.string)
            for feature_name in self.feature_names
        })

    # read ground truth labels
    labels = (tf.cast(
        tf.sparse_to_dense(contexts["labels"].values, (self.num_classes,), 1,
            validate_indices=False),
        tf.bool))

    # loads (potentially) different types of features and concatenates them
    num_features = len(self.feature_names)
    assert num_features > 0, "No feature selected: feature_names is empty!"

    assert len(self.feature_names) == len(self.feature_sizes), \
    "length of feature_names (={}) != length of feature_sizes (={})".format( \
    len(self.feature_names), len(self.feature_sizes))

    num_frames = -1  # the number of frames in the video
    feature_matrices = [None] * num_features  # an array of different features
    for feature_index in range(num_features):
      feature_matrix, num_frames_in_this_feature = self.get_video_matrix(
          features[self.feature_names[feature_index]],
          self.feature_sizes[feature_index],
          self.max_frames,
          max_quantized_value,
          min_quantized_value)
      if num_frames == -1:
        num_frames = num_frames_in_this_feature
      else:
        tf.assert_equal(num_frames, num_frames_in_this_feature)

      feature_matrices[feature_index] = feature_matrix

    # cap the number of frames at self.max_frames
    num_frames = tf.minimum(num_frames, self.max_frames)

    # concatenate different features
    video_matrix = tf.concat(feature_matrices, 1)

    # convert to batch format.
    # TODO: Do proper batch reads to remove the IO bottleneck.
    batch_video_ids = tf.expand_dims(contexts["video_id"], 0)
    batch_video_matrix = tf.expand_dims(video_matrix, 0)
    batch_labels = tf.expand_dims(labels, 0)
    batch_frames = tf.expand_dims(num_frames, 0)

    return batch_video_ids, batch_video_matrix, batch_labels, batch_frames

In [ ]:
prepare_serialized_examples(self, serialized_example,max_quantized_value=2, min_quantized_value=-2)


In [ ]:
list_mean_audio=list(tf_example.features.feature['mean_audio'].float_list.value)
list_mean_audio


In [ ]:
len(list_mean_audio)

In [ ]:
list_mean_rgb=list(tf_example.features.feature['mean_rgb'].float_list.value)
list_mean_rgb

In [ ]:
len(list_mean_rgb)

In [ ]:
for example in tf.python_io.tf_record_iterator("../input/video_level/train-1.tfrecord"):
    total_sample_counter += 1
    tf_example2 = tf.train.Example.FromString(example)
tf_example2

In [ ]:
for example in tf.python_io.tf_record_iterator("../input/video_level/train-2.tfrecord"):
    total_sample_counter += 1
    tf_example3 = tf.train.Example.FromString(example)
tf_example3

In [ ]:
for example in tf.python_io.tf_record_iterator("../input/video_level/train-7.tfrecord"):
    total_sample_counter += 1
    tf_example8 = tf.train.Example.FromString(example)
tf_example8

In [ ]:
label_example = list(tf_example1.features.feature['labels'].int64_list.value)
label_example

In [ ]:
len(label_example)

In [ ]:
label_counts.append(len(label_example))
labels = labels + label_example
label_example_textual = list(labels_df[labels_df['label_id'].isin(label_example)]['label_name'])

In [ ]:
label_counts

In [ ]:
labels

In [ ]:
labels_df['label_name']

In [ ]:
label_example_textual 

In [ ]:
set(label_example_textual)

In [ ]:
textual_labels_nested

In [ ]:
textual_labels

In [ ]:
pd.DataFrame({'label': textual_labels}).groupby('label').size()

In [ ]:
def grouped_data_for(l):
    # wrap the grouped data into dataframe, since the inner is pd.Series, not what we need
    l_with_c = pd.DataFrame(
        pd.DataFrame({'label': l}).groupby('label').size().rename('n')
    ).sort_values('n', ascending=False).reset_index()
    return l_with_c

In [ ]:
N = 20

textual_labels_with_counts_all = grouped_data_for(textual_labels)
textual_labels_with_counts_all

In [ ]:
type(textual_labels_with_counts_all['label'][0:50].values)

In [ ]:
top_50_labels = list(textual_labels_with_counts_all['label'][0:50].values)
top_50_labels

In [ ]:
easy_install youtubevideo

In [ ]:

def play_one_vid(record_name, video_index):
    return vid_ids[video_index]
    
# this worked on my local jupyter notebook, but doesn't show on kaggle kernels:
YouTubeVideo(play_one_vid(example, 7))

In [ ]:
import tensorflow as tf
import numpy as np
from IPython.display import YouTubeVideo


video_lvl_record = "../input/video_level/train-1.tfrecord"
frame_lvl_record = "../input/frame_level/train-1.tfrecord"

In [ ]:
vid_ids = []
labels = []
mean_rgb = []
mean_audio = []

for example in tf.python_io.tf_record_iterator(video_lvl_record):
    tf_example = tf.train.Example.FromString(example)

    vid_ids.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8'))
    labels.append(tf_example.features.feature['labels'].int64_list.value)
    mean_rgb.append(tf_example.features.feature['mean_rgb'].float_list.value)
    mean_audio.append(tf_example.features.feature['mean_audio'].float_list.value)

In [ ]:
for example in tf.python_io.tf_record_iterator(video_lvl_record):
    tf_example = tf.train.Example.FromString(example)
tf_example

In [ ]:
n=10
from collections import Counter
label_mapping = pd.Series.from_csv('../input/label_names.csv',header=0).to_dict()

top_n = Counter([item for sublist in labels for item in sublist]).most_common(n)
top_n_labels = [int(i[0]) for i in top_n]
top_n_label_names = [label_mapping[x] for x in top_n_labels]
top_n_label_names

In [ ]:
type(top_n_label_names)

In [ ]:
list(labels)

In [ ]:

import networkx as nx
from itertools import combinations

G=nx.Graph()

G.clear()
for list_of_nodes in labels:
    filtered_nodes = set(list_of_nodes).intersection(set(top_n_labels))  
    for node1,node2 in list(combinations(filtered_nodes,2)): 
        node1_name = label_mapping[node1]
        node2_name = label_mapping[node2]
        G.add_node(node1_name)
        G.add_node(node2_name)
        G.add_edge(node1_name, node2_name)

nx.draw_networkx(G)

In [ ]:
filtered_nodes = set(list_of_nodes).intersection(set(top_n_labels)) 
filtered_nodes
set(list_of_nodes)

In [ ]:
count=0
num=0
for list_of_nodes in labels:
    filtered_nodes = set(list_of_nodes).intersection(set(top_n_labels))  
    print (len(list(combinations(filtered_nodes,3))))
    print ('abc', num)
    num+=1
    for node1 in list(combinations(filtered_nodes,3)): 
        print('nodes', node1, node2)
        count+=1
print (count)        
    

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from IPython.display import YouTubeVideo
from subprocess import check_output

labels = pd.read_csv('../input/label_names.csv') #4716 labels

cols_rgb = ["rgb_{}".format(i) for i in range(1024)]
cols_aud = ["aud_{}".format(i) for i in range(128)]
cols_y   = ["y_{}".format(i) for i in range(4716 + 1)]

usevidlvl = True
usefralvl = False

In [ ]:
#PREPROCESSING

#homemade "dataframfy" method
def dffy(filenames):
    index = []
    rgb = []
    aud = []
    y = []
    if len(filenames)>0:
        isVideo = filenames[0].find("video")!=-1

    for filename in filenames:
        for example in tf.python_io.tf_record_iterator(filename):
            tf_example = tf.train.Example.FromString(example)
            index.append(tf_example.features.feature['video_id'].bytes_list.value[0].decode(encoding='UTF-8'))
            if isVideo:
                rgb.append(tf_example.features.feature['mean_rgb'].float_list.value)
                aud.append(tf_example.features.feature['mean_audio'].float_list.value)
            else:
                rgb.append(tf_example.features.feature['rgb'].float_list.value)
                aud.append(tf_example.features.feature['audio'].float_list.value)
            y.append(np.array(tf_example.features.feature['labels'].int64_list.value))
            
    index = pd.DataFrame(np.array(index),columns=["id"])
    rgb = pd.DataFrame(np.array(rgb),columns=cols_rgb)
    aud = pd.DataFrame(np.array(aud),columns=cols_aud)
    y2 = pd.DataFrame(np.zeros((len(index),len(labels) + 1)))
    for i in range(len(y)):
        y2.loc[i,y[i].clip(0,len(labels))] = 1
    y2.columns=cols_y
    y = []
    
    return [index,rgb,aud,y2]

In [ ]:
if usevidlvl:
    vidfilenames = ["../input/video_level/train-{}.tfrecord".format(i) for i in range(10)]
    [vid_id,vid_rgb,vid_aud,vid_y] = dffy(vidfilenames)

if usefralvl:
    frafilenames = ["../input/frame_level/train-{}.tfrecord".format(i) for i in range(10)]
    [fra_id,fra_rgb,fra_aud,fra_y] = dffy(frafilenames)

In [ ]:
[vid_id,vid_rgb,vid_aud,vid_y]

In [ ]:
vid_rgb

In [ ]:
vid_aud

In [ ]:
vid_aud.shape

In [ ]:
vid_y

In [ ]:
usefralvl = True
if usefralvl:
    frafilenames = ["../input/frame_level/train-{}.tfrecord".format(i) for i in range(10)]
    [fra_id,fra_y] = dffy(frafilenames)

In [ ]:
fra_id